<a href="https://colab.research.google.com/github/Haripriya-AIML/Priya/blob/main/AIML_A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement Logistic Regression Model Using Titanic Ship Dataset

Part 1 - Import the required Python, Pandas, Matplotlib, Seaborn packages.
[CO2]
1. Load the Titanic Ship Passengers dataset into a dataframe using pandas
2. Check the data types of each feature(column) in the dataset.
3. Generate a summary of the dataset for min, max, stddev,
quartile vales for 25%,50%,75%,90%,
4. List the names of columns/features in the dataset
5. Perform Exploratory analysis - plot numeric features, check relative size of survived/un-
survived, check if any pattern on gender. passenger class, class-wise survival rate, sibil-
ings, overall age distribution, class-wise age distribution - apply bar plot, histogram,
box plots to visualize.
6. Perform data wrangling - impute age data, drop unnecessary features, recode the cate-
gorical features to a class